# Looking up the Tables:

In [265]:
# importing module
import sqlite3

# connecting to the database
connection = sqlite3.connect("/content/drive/MyDrive/Colab Notebooks/sqlite/transactions.db")

In [266]:
# print records of Devices table
def devices_table():
  sql_command = """select * from Devices;"""
  crsr = connection.cursor()
  crsr.execute(sql_command)
  ans = crsr.fetchall()

  num_fields = len(crsr.description)
  field_names = [i[0] for i in crsr.description]
  print(field_names)
  for i in ans: 
    print(i)

  crsr.close()

devices_table()

['id', 'device_name']
(1, 'Desktop')
(2, 'Tablet')
(3, 'Mobile Phone')
(4, 'Unknown')


In [284]:
# print first 10 records of Transaction table
def transaction_table():
  sql_command = """select * from Transactions;"""
  crsr = connection.cursor()
  crsr.execute(sql_command)
  ans = crsr.fetchall()

  num_fields = len(crsr.description)
  field_names = [i[0] for i in crsr.description]
  print(field_names)
  # print(', '.join(str(x) for x in field_names))
  for i in ans[:20]: 
    print(i)

  crsr.close()

transaction_table()

['id', 'datetime', 'visitor_id', 'device_type', 'revenue', 'tax']
(1, '2019-10-05 00:00:00', 1016870803637, 1, None, 0.19)
(2, '2019-10-11 00:00:00', 48076008851343, 4, 0.0173789153676843, 0.07)
(3, '2019-08-19 00:00:00', 72300317870324, 2, 0.0020015926250121978, 0.07)
(4, '2019-10-14 00:00:00', 84516511452821, 3, 0.012191527080364702, 0.19)
(5, '2019-09-10 00:00:00', 44780848857001, 3, 0.02792511164247113, 0.07)
(6, '2019-09-20 00:00:00', 54145933160851, 2, 0.0015068945344001408, 0.07)
(7, '2019-10-13 00:00:00', 68564288760762, 4, None, 0.19)
(8, '2019-10-22 00:00:00', 74593067572606, 2, 0.001174484066488895, 0.07)
(9, '2019-08-12 00:00:00', 74912374801029, 4, 0.0010201338790432155, 0.07)
(10, '2019-10-25 00:00:00', 67206600864292, 4, 0.0011203940531355958, 0.07)
(11, '2019-10-28 00:00:00', 72257448527137, 4, 0.006003262242931342, 0.19)
(12, '2019-08-16 00:00:00', 31161783828987, 2, 0.07379172341645311, 0.07)
(13, '2019-09-21 00:00:00', 74019332491246, 3, None, 0.07)
(14, '2019-10-19 

# Task 1:
Write a Python script to find out which visitor created the most revenue.

In [270]:
sql_command = """select visitor_id, round(MAX(revenue),2) as max_revenue from Transactions;"""
crsr = connection.cursor()
crsr.execute(sql_command)

columns = crsr.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in crsr.fetchall()]
crsr.close()

print(result)

[{'visitor_id': 47447038460279, 'max_revenue': 999.87}]


# Task 2: 
Write a Python script to find out on which day most revenue for users who ordered via a mobile phone was created.

In [271]:
# The below query works too
# sql_command = """select datetime, visitor_id, device_type, MAX(revenue) as max_revenue_mobilephone from Transactions\
#  where device_type=(select id from Devices where device_name='Mobile Phone');"""

sql_command = """select strftime('%d/%m/%Y',t.datetime) as day, t.visitor_id, d.device_name, round(MAX(t.revenue),2) as max_revenue from Transactions as t \
INNER JOIN Devices as d ON t.device_type = d.id \
group by t.device_type \
having d.device_name='Mobile Phone';"""

crsr = connection.cursor()
crsr.execute(sql_command)

columns = crsr.description 
result = [{columns[index][0]:column for index, column in enumerate(value)} for value in crsr.fetchall()]
crsr.close()

print(result)

[{'day': '20/09/2019', 'visitor_id': 22533704138974, 'device_name': 'Mobile Phone', 'max_revenue': 997.93}]


# Task 3: 
Write a Python script that combines the contents of Devices and Transactions and store it as a single flat file including the column names.

In [273]:
sql_command = """select t.id, t.device_type, d.device_name, t.datetime, t.visitor_id,  t.revenue, t.tax  from Transactions as t INNER JOIN Devices as d ON t.device_type = d.id;"""
crsr = connection.cursor()
crsr.execute(sql_command)
ans = crsr.fetchall()

field_names = [i[0] for i in crsr.description]
# print(field_names)

field_names = [tuple(field_names)]
final = field_names + ans
print("rows in final table:",len(final)) #should be 10k+1 including the column names

with open('joined_tables.txt', 'w') as f:
    for row in final:
        # print row
        f.write("%s\n" % str(', '.join(str(x) for x in row)))

print("File is saved as: 'joined_tables.txt'")
crsr.close()

rows in final table: 10001
File is saved as: 'joined_tables.txt'


# Task 4:
As stated in the SQL comments the created revenue is currently stored in USD.
Update the data stored in the database to have the created revenue in EUR.

In [274]:
# Reading the eurofxref-hist-90d.xml file
from xml.etree import ElementTree

def indent(elem, level=0):
    i = "\n" + level*"  "
    j = "\n" + (level-1)*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for subelem in elem:
            indent(subelem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = j
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = j
    return elem        

root = ElementTree.parse('/content/eurofxref-hist-90d.xml').getroot()
indent(root)
ElementTree.dump(root[2][0])
# print(root[2][0].attrib["time"])
print(root[2][0][0].attrib['rate'])

<ns0:Cube xmlns:ns0="http://www.ecb.int/vocabulary/2002-08-01/eurofxref" time="2019-11-01">
      <ns0:Cube currency="USD" rate="1.1139" />
    <ns0:Cube currency="JPY" rate="120.43" />
    <ns0:Cube currency="BGN" rate="1.9558" />
    <ns0:Cube currency="CZK" rate="25.514" />
    <ns0:Cube currency="DKK" rate="7.4712" />
    <ns0:Cube currency="GBP" rate="0.86008" />
    <ns0:Cube currency="HUF" rate="328.33" />
    <ns0:Cube currency="PLN" rate="4.2535" />
    <ns0:Cube currency="RON" rate="4.7547" />
    <ns0:Cube currency="SEK" rate="10.6993" />
    <ns0:Cube currency="CHF" rate="1.1013" />
    <ns0:Cube currency="ISK" rate="138.1" />
    <ns0:Cube currency="NOK" rate="10.1638" />
    <ns0:Cube currency="HRK" rate="7.46" />
    <ns0:Cube currency="RUB" rate="71.0786" />
    <ns0:Cube currency="TRY" rate="6.3761" />
    <ns0:Cube currency="AUD" rate="1.6151" />
    <ns0:Cube currency="BRL" rate="4.4437" />
    <ns0:Cube currency="CAD" rate="1.4682" />
    <ns0:Cube currency="CNY" ra

In [275]:
# Creating dictionary of dates and the resepective conversion rate from usd to euro
usd_to_euro_rates = {}
for type_tag in root.findall('{http://www.ecb.int/vocabulary/2002-08-01/eurofxref}Cube/'):
    # print(type_tag.tag)
    for tag in type_tag:
      if tag.get('currency')=='USD':
        usd_to_euro_rates[type_tag.get('time')] = 1/float(tag.get('rate'))

In [277]:
list(usd_to_euro_rates.items())[:2]

[('2019-11-01', 0.8977466558937068), ('2019-10-31', 0.8965393580778197)]

In [281]:
# Getting all the dates from the transaction table
sql_command_read = """select strftime('%Y-%m-%d', datetime) as day from Transactions;"""
crsr = connection.cursor()
crsr.execute(sql_command_read)
ans = crsr.fetchall()
crsr.close()

# Iterating through all the dates collected above and updating the respective revenue
# Setting the revenue to NULL for the dates whose conversion rate is not available in the xml file
for date in ans:
  date_f = ', '.join(str(x) for x in date)  # converting the date to required format
  if date_f in usd_to_euro_rates.keys():
    euro_rate = usd_to_euro_rates[date_f]
  else:
    euro_rate = -1    # For the dates whoch are not in the xml file

  crsr = connection.cursor()

  if euro_rate < 0:
    sql_command_update = """UPDATE Transactions SET revenue=NULL WHERE strftime('%Y-%m-%d', datetime)=:1;"""
    val = (date_f,)   
  else:
    sql_command_update = """UPDATE Transactions SET revenue=revenue * :1 WHERE strftime('%Y-%m-%d', datetime)=:2;"""
    val = (euro_rate, date_f)
  
  crsr.execute(sql_command_update, val)
  crsr.close()

connection.commit() #commit

In [285]:
transaction_table() # display the updated table

['id', 'datetime', 'visitor_id', 'device_type', 'revenue', 'tax']
(1, '2019-10-05 00:00:00', 1016870803637, 1, None, 0.19)
(2, '2019-10-11 00:00:00', 48076008851343, 4, 0.0173789153676843, 0.07)
(3, '2019-08-19 00:00:00', 72300317870324, 2, 0.0020015926250121978, 0.07)
(4, '2019-10-14 00:00:00', 84516511452821, 3, 0.012191527080364702, 0.19)
(5, '2019-09-10 00:00:00', 44780848857001, 3, 0.02792511164247113, 0.07)
(6, '2019-09-20 00:00:00', 54145933160851, 2, 0.0015068945344001408, 0.07)
(7, '2019-10-13 00:00:00', 68564288760762, 4, None, 0.19)
(8, '2019-10-22 00:00:00', 74593067572606, 2, 0.001174484066488895, 0.07)
(9, '2019-08-12 00:00:00', 74912374801029, 4, 0.0010201338790432155, 0.07)
(10, '2019-10-25 00:00:00', 67206600864292, 4, 0.0011203940531355958, 0.07)
(11, '2019-10-28 00:00:00', 72257448527137, 4, 0.006003262242931342, 0.19)
(12, '2019-08-16 00:00:00', 31161783828987, 2, 0.07379172341645311, 0.07)
(13, '2019-09-21 00:00:00', 74019332491246, 3, None, 0.07)
(14, '2019-10-19 

In [ ]:
connection.close() # closing the connection

## Task 5:
Imagine you have to add support for other DBMS, how would you address this
request? Write a Python script that exemplarily uses PostgreSQL.

*  For this task, I have taken help from google and tried to write down whatever I could get or think of.

To support different DBMS, we will have to make different connections and cursors ready. For example, if we want to add support for MySQL or PostgreSQL, the drivers related to them have to be kept ready and also, the queries have to be migrated. At any cost, we cannot keep the same queries for each DBMS and has to be changed based on the selected database. For this kind of requirement, there's a library based on Python called **PyQt**:

*   Use PyQt’s SQL support to reliably connect to a database
*   Execute SQL queries on a database using PyQt
*   Use PyQt’s Model-View architecture in database applications
*   Display and edit data using different PyQt widgets

In PyQt, you can create a database connection by using the QSqlDatabase class. This class represents a connection and provides an interface for accessing the database. To create a connection, just call .addDatabase() on QSqlDatabase.


```
QSqlDatabase.addDatabase(
    driver, connectionName=QSqlDatabase.defaultConnection
)
```



A call to .addDatabase() adds a database connection to a list of available connections. This list is a global registry that PyQt maintains behind the scenes to keep track of the available connections in an application. Registering your connections with a meaningful connectionName will allow you to manage several connections in a database application.

Besides, I would like to follow below video and some related videos too to find few more alternatives.

**References:**
* How to connect with multiple databases instances using Python Programming? - https://www.youtube.com/watch?v=uRT76V45CWo&ab_channel=IamPython

* Something like below which I had thought too and found on internet where the common fuctions can be kept in one and then connector and different DBMS based on call: - https://stackoverflow.com/questions/28871693/design-a-connection-to-multiple-databases-in-python

```
class Connector(Database):
    """This is a super class, common elements go here"""
    def __init__(self):
        self.sql_search = "SELECT ID FROM TAG WHERE DATA = %s"
        self.common_varialbe = None #placeholder
        Database.__init__(self) #add necessary arguments

    def _get_tag(self, tagcipher, wildcard):
        #replace the sql search string with the wildcard.
        self._cur.execute(self.sql_search % (wildcard) , ([tagcipher]))
        rv = self._cur.fetchone()
        return rv
   def some_common_method(self, uncommon_value):
       self.common_variable = uncommon_value

class Postgresql(Connector):
    """postgresql subclass using %s. 
    unique postgresql elements go here"""
    def __init__(self):
        #initialise the superclass
        Connector.__init__(self)

        self.wildcard = '%s'
        self.uncommon_value = 'py hole'
        #other unique values go here

class Sqlite(Connector):
    """etc"""
    def __init__(self):
        #initialise the superclass
        Connector.__init__(self)

        self.wildcard = '?'
        #other unique values go here

    #other methods
```





---
For a Python script in PostgreSQl, we use the libarary- "psycopg2".
The postgreSQl is written in another file (**PsotgreSQL_python.ipynb**) as I had installed on local and tried connecting with the credentials.

Several tables are created in that after creating the connection with the database and then the regular operations of CRUD has been performed like insert and update, and then closed the connection.
